### GRASS GIS Raster Calculator in Loop ###

In [ ]:
from gasp3.pyt.oss import lst_ff

inFld = r'D:\indev\matereo\all_bands_10m_2006'
ouFld = r'D:\indev\matereo\water_bands_2006'
water = r'D:\indev\matereo\masks_1805\main_water.tif'

In [ ]:
bnds = lst_ff(inFld, file_format='.tif')
print(bnds)

In [ ]:
import os
from gasp3.gt.prop.prj import get_epsg_raster
from gasp3.gt.wenv.grs import run_grass

# Get EPSG from refRaster
epsg = get_epsg_raster(water, returnIsProj=None)
    
"""
Start GRASS GIS Session
"""
grsb = run_grass(
    inFld, grassBIN='grass77', location='grs_work',
    srs=epsg
)
    
import grass.script as grass
import grass.script.setup as gsetup
    
gsetup.init(grsb, inFld, 'grs_work', 'PERMANENT')

from gasp3.pyt.oss       import fprop
from gasp3.gt.torst     import rst_to_grs, grs_to_rst
from gasp3.gt.wenv.grs   import rst_to_region
from gasp3.gt.spnlst.alg import rstcalc

main = rst_to_grs(water, fprop(water, 'fn'))
rst_to_region(main)

# Add bands to GRASS GIS
__bnds = [rst_to_grs(i, fprop(i, 'fn')) for i in bnds]

# Raster calculator
calc_bnd = [rstcalc("{} * {}".format(i, main), 'rsl_{}'.format(i), api='pygrass') for i in __bnds]

# Export bands
for i in range(len(calc_bnd)):
    grs_to_rst(calc_bnd[i], os.path.join(ouFld, __bnds[i] + '.tif'))

In [ ]:
"""
Raster's to Mosaic
"""

from gasp.gt.nop.mos import rsts_to_mosaic

output = r'C:\gwork\fireloc\tst_windninja\mdt_wind.tif'

rsts_to_mosaic([
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_75.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_76.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_77.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_104.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_105.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_106.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_133.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_134.tif',
    r'C:\gwork\fireloc\datasets\mdt_m888\dem_f\dem_135.tif',
], output, api="rasterio", fformat='.tif')

In [ ]:
"""
Principal Component Analysis

refs:
https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
https://towardsdatascience.com/a-step-by-step-explanation-of-principal-component-analysis-b836fb9c97e2
"""

bands = [
    r'D:\indev\matereo\water_bands_2804\T33WWR_20190428T104029_B02_10m.tif',
    r'D:\indev\matereo\water_bands_2804\T33WWR_20190428T104029_B03_10m.tif',
    r'D:\indev\matereo\water_bands_2804\T33WWR_20190428T104029_B04_10m.tif',
    r'D:\indev\matereo\water_bands_2804\T33WWR_20190428T104029_B08_10m.tif',
    r'D:\indev\matereo\water_bands_1805\T33WWR_20190518T104029_B02_10m.tif',
    r'D:\indev\matereo\water_bands_1805\T33WWR_20190518T104029_B03_10m.tif',
    r'D:\indev\matereo\water_bands_1805\T33WWR_20190518T104029_B04_10m.tif',
    r'D:\indev\matereo\water_bands_1805\T33WWR_20190518T104029_B08_10m.tif'
]

In [ ]:
from osgeo import gdal, gdal_array
import numpy as np

In [ ]:
# Open Images

img_src = [gdal.Open(i, gdal.GA_ReadOnly) for i in bands]

ndVal = img_src[0].GetRasterBand(1).GetNoDataValue()

img_x = np.zeros((
    img_src[0].RasterYSize, img_src[0].RasterXSize, len(img_src)),
    gdal_array.GDALTypeCodeToNumericTypeCode(
        img_src[0].GetRasterBand(1).DataType
    )
)

for b in range(img_x.shape[2]):
    img_x[:, :, b] = img_src[b].GetRasterBand(1).ReadAsArray()

In [ ]:
new_shape = (img_x.shape[0] * img_x.shape[1], img_x.shape[2])
X = img_x[:, :, :8].reshape(new_shape)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=X.shape[1])

X_pca = pca.fit_transform(X)

#PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    #svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
X_pca = X_pca.reshape(img_x[:, :, :].shape)

In [ ]:
pca_array = [np.zeros((X_pca.shape[0], X_pca.shape[1]), X_pca.dtype) for i in range(X_pca.shape[2])]

for i in range(X_pca.shape[2]):
    pca_array[i][:, :] = X_pca[:, :, i]

In [ ]:
from gasp3.gt.torst import obj_to_rst
import os
outFolder = r'D:\indev\matereo\pca_tst'
tmp = img_src[0].GetRasterBand(1).ReadAsArray()

for i in range(len(pca_array)):
    nd_nd = np.amin(pca_array[i]) - 1
    
    np.place(pca_array[i], tmp==ndVal, nd_nd)
    
    obj_to_rst(
        pca_array[i],
        os.path.join(outFolder, 'pca_{}.tif'.format(i+1)),
        bands[0], noData=nd_nd
    )

In [ ]:
print(np.amin(pca_array[0]))

In [ ]:
print(X_pca.shape)
print(X_pca)

In [ ]:
print(img_x.shape)
print(img_x)

In [ ]:
# IMG to Array

from gasp3.gt.fmrst import rst_to_array

_arr = [rst_to_array(b) for b in bands]

In [ ]:
from rasterio.crs import CRS

crs = CRS.from_epsg(3763)
#print(crs.to_proj4())

In [ ]:
print(out_meta)